In [1]:
!pip install pandas scikit-learn biopython lazypredict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and

In [2]:
import pandas as pd
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from lazypredict.Supervised import LazyRegressor
from scipy.stats import pearsonr
import numpy as np

# Step 1: Load dataset
data = pd.read_csv('/content/PDB+TM+FASTA (1).csv')

# Step 2: Extract features using biopython
def extract_biopython_features(sequence):
    try:
        sequence = sequence.replace('\r', '').replace('\n', '')
        analysis = ProteinAnalysis(sequence)
        features = {
            'molecular_weight': analysis.molecular_weight(),
            'aromaticity': analysis.aromaticity(),
            'instability_index': analysis.instability_index(),
            'isoelectric_point': analysis.isoelectric_point(),
            'gravy': analysis.gravy(),
            'flexibility': sum(analysis.flexibility()) / len(sequence),
            'charge_at_pH_7': analysis.charge_at_pH(7.0),
        }
        return features
    except Exception as e:
        print(f"Error processing sequence: {sequence}. Error: {e}")
        return None

sequences = data['Sequence'].tolist()
biopython_features = [extract_biopython_features(seq) for seq in sequences]
biopython_features = [feat for feat in biopython_features if feat is not None]  # Remove None entries
biopython_features_df = pd.DataFrame(biopython_features)

# Step 3: Ensure the length of the sequences matches the length of the melting_temp
if len(biopython_features_df) != len(data['melting_temp']):
    raise ValueError("Mismatch between number of sequences and melting temperatures")

# Step 4: Combine features with stability values (as continuous values)
combined_df = pd.concat([biopython_features_df, data['melting_temp'].iloc[:len(biopython_features_df)]], axis=1)

# Step 5: Split data into training and testing sets
X = combined_df.drop(columns=['melting_temp']).values
y = combined_df['melting_temp'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 7: Apply LazyPredict for model evaluation
reg = LazyRegressor(ignore_warnings=True, random_state=42)
models, predictions = reg.fit(X_train_scaled, X_test_scaled, y_train, y_test)

# Check shapes to debug
print("Shapes: ", X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape)
print("Predictions shape: ", predictions.shape)

# Step 8: Calculate Pearson correlation coefficient
pearson_results = {}
for model in predictions.columns:
    if len(predictions[model]) != len(y_test):
        print(f"Skipping model {model} due to length mismatch.")
        continue
    pearson_corr, _ = pearsonr(y_test, predictions[model])
    pearson_results[model] = pearson_corr

# Step 9: Display Pearson correlation coefficients
for model, pearson_corr in pearson_results.items():
    print(f"Pearson correlation for {model}: {pearson_corr:.4f}")

# Step 10: Display model performance with Pearson correlation
models['Pearson Correlation'] = models.index.map(pearson_results.get)
print(models)

# Step 11: Select the best performing model based on Pearson correlation
if models['Pearson Correlation'].isnull().all():
    print("No valid Pearson correlations calculated. Check for issues in model predictions.")
else:
    best_model = models.loc[models['Pearson Correlation'].idxmax()]
    print("\nBest model based on Pearson correlation coefficient:")
    print(best_model)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
100%|██████████| 42/42 [00:04<00:00,  9.18it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000146 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 657
[LightGBM] [Info] Number of data points in the train set: 279, number of used features: 7
[LightGBM] [Info] Start training from score 64.132101
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [3]:
import pandas as pd
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import numpy as np

# Load dataset
data = pd.read_csv('/content/PDB+TM+FASTA (1).csv')

# Extract features using biopython
def extract_biopython_features(sequence):
    try:
        sequence = sequence.replace('\r', '').replace('\n', '')
        analysis = ProteinAnalysis(sequence)
        features = {
            'molecular_weight': analysis.molecular_weight(),
            'aromaticity': analysis.aromaticity(),
            'instability_index': analysis.instability_index(),
            'isoelectric_point': analysis.isoelectric_point(),
            'gravy': analysis.gravy(),
            'flexibility': sum(analysis.flexibility()),
            'charge_at_pH_7': analysis.charge_at_pH(7.0),
            'amino_acid_comp': analysis.count_amino_acids()
        }
        return features
    except Exception as e:
        print(f"Error processing sequence: {sequence}. Error: {e}")
        return None

sequences = data['Sequence'].tolist()
biopython_features = [extract_biopython_features(seq) for seq in sequences]
biopython_features = [feat for feat in biopython_features if feat is not None]

# Convert amino acid composition dicts to separate columns
amino_acids = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
for feature in biopython_features:
    for aa in amino_acids:
        feature[f'aa_{aa}'] = feature['amino_acid_comp'].get(aa, 0)
    del feature['amino_acid_comp']

biopython_features_df = pd.DataFrame(biopython_features)

# Ensure the length of the sequences matches the length of the melting_temp
if len(biopython_features_df) != len(data['melting_temp']):
    raise ValueError("Mismatch between number of sequences and melting temperatures")

# Combine features with stability values
combined_df = pd.concat([biopython_features_df, data['melting_temp'].iloc[:len(biopython_features_df)]], axis=1)

# Remove outliers (optional, depends on your dataset)
def remove_outliers(df, threshold=3):
    z_scores = np.abs((df - df.mean()) / df.std())
    return df[(z_scores < threshold).all(axis=1)]

combined_df = remove_outliers(combined_df)

# Split data into training and testing sets
X = combined_df.drop(columns=['melting_temp']).values
y = combined_df['melting_temp'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Hyperparameter tuning for RandomForestRegressor and GradientBoostingRegressor
param_grid_rf = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [None, 10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__bootstrap': [True, False]
}

param_grid_gb = {
    'gb__n_estimators': [100, 200, 300],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 5, 7],
    'gb__min_samples_split': [2, 5, 10],
    'gb__min_samples_leaf': [1, 2, 4]
}

pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42))
])

pipeline_gb = Pipeline([
    ('scaler', StandardScaler()),
    ('gb', GradientBoostingRegressor(random_state=42))
])

grid_search_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2)
grid_search_gb = GridSearchCV(estimator=pipeline_gb, param_grid=param_grid_gb, cv=5, n_jobs=-1, verbose=2)

# Train the models and find the best parameters
grid_search_rf.fit(X_train, y_train)
grid_search_gb.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
best_gb = grid_search_gb.best_estimator_

# Make predictions with the best models
y_train_pred_rf = best_rf.predict(X_train)
y_test_pred_rf = best_rf.predict(X_test)
y_train_pred_gb = best_gb.predict(X_train)
y_test_pred_gb = best_gb.predict(X_test)

# Calculate accuracy metrics for both models
train_rmse_rf = mean_squared_error(y_train, y_train_pred_rf, squared=False)
test_rmse_rf = mean_squared_error(y_test, y_test_pred_rf, squared=False)
train_r2_rf = r2_score(y_train, y_train_pred_rf)
test_r2_rf = r2_score(y_test, y_test_pred_rf)

train_rmse_gb = mean_squared_error(y_train, y_train_pred_gb, squared=False)
test_rmse_gb = mean_squared_error(y_test, y_test_pred_gb, squared=False)
train_r2_gb = r2_score(y_train, y_train_pred_gb)
test_r2_gb = r2_score(y_test, y_test_pred_gb)

print(f"Random Forest Best Parameters: {grid_search_rf.best_params_}")
print(f"Random Forest Training RMSE: {train_rmse_rf}")
print(f"Random Forest Testing RMSE: {test_rmse_rf}")
print(f"Random Forest Training R²: {train_r2_rf}")
print(f"Random Forest Testing R²: {test_r2_rf}")

print(f"Gradient Boosting Best Parameters: {grid_search_gb.best_params_}")
print(f"Gradient Boosting Training RMSE: {train_rmse_gb}")
print(f"Gradient Boosting Testing RMSE: {test_rmse_gb}")
print(f"Gradient Boosting Training R²: {train_r2_gb}")
print(f"Gradient Boosting Testing R²: {test_r2_gb}")

# Cross-validation scores for more reliable performance estimation
cv_rf = cross_val_score(best_rf, X, y, cv=5, scoring='r2')
cv_gb = cross_val_score(best_gb, X, y, cv=5, scoring='r2')

print(f"Random Forest Cross-Validation R² Scores: {cv_rf}")
print(f"Gradient Boosting Cross-Validation R² Scores: {cv_gb}")
print(f"Random Forest Average Cross-Validation R² Score: {np.mean(cv_rf)}")
print(f"Gradient Boosting Average Cross-Validation R² Score: {np.mean(cv_gb)}")


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Random Forest Best Parameters: {'rf__bootstrap': True, 'rf__max_depth': None, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Random Forest Training RMSE: 5.421423118832182
Random Forest Testing RMSE: 11.671268749527682
Random Forest Training R²: 0.9214679051614564
Random Forest Testing R²: 0.5988893004553472
Gradient Boosting Best Parameters: {'gb__learning_rate': 0.2, 'gb__max_depth': 3, 'gb__min_samples_leaf': 1, 'gb__min_samples_split': 2, 'gb__n_estimators': 100}
Gradient Boosting Training RMSE: 1.6374933338122923
Gradient Boosting Testing RMSE: 11.822831406202905
Gradient Boosting Training R²: 0.9928356027771751
Gradient Boosting Testing R²: 0.5884040421719319
Random Forest Cross-Validation R² Scores: [  0.1557677  -10.75377857  -1.5015393   -1.80851438 -25.42892199]
Gradient Boosting Cross-Validation R² Scores: [  0.12798

In [5]:
# Import necessary libraries
import joblib
import pandas as pd



# Save the trained model
joblib.dump(best_rf, '/content/random_forest_model.pkl')

# Predicting on the test set
y_pred = best_rf.predict(X_test)

# Generate a CSV file with actual and predicted values
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results_df.to_csv('/content/actual_vs_predicted.csv', index=False)


In [11]:
def extract_features_from_pdb(pdb_file):


    features = []


    print("Processing PDB file:", pdb_file)


    for i in range(27):
        try:
            feature = process_pdb(pdb_file, feature_name=f'feature{i+1}')
            features.append(feature)
            # Debugging: Print each feature after extraction
            print(f"Extracted feature{i+1}:", feature)
        except Exception as e:
            print(f"Error extracting feature{i+1}: {e}")
            features.append(np.nan)  # Handle missing features gracefully

    return features

# Define your `process_pdb` function
def process_pdb(pdb_file, feature_name):

    return np.random.rand()

# Load the saved model
loaded_model = joblib.load('random_forest_model.pkl')

# Extract features from the new PDB file
new_pdb_file = '/content/6ezq.pdb'
new_features = extract_features_from_pdb(new_pdb_file)

# Convert new_features to a numeric array and ensure it is a 2D array
import numpy as np

new_features = np.array(new_features, dtype=np.float32).reshape(1, -1)

# Debugging: Print new_features to identify NaN values
print("Extracted features:", new_features)

if np.isnan(new_features).any():
    raise ValueError("The extracted features contain NaN values. Please check the feature extraction process.")

# Predict the melting temperature
predicted_melting_temp = loaded_model.predict(new_features)
print(f"Predicted Melting Temperature: {predicted_melting_temp[0]}")


Processing PDB file: /content/6ezq.pdb
Extracted feature1: 0.5017757005426897
Extracted feature2: 0.9466564318248974
Extracted feature3: 0.6599164479149328
Extracted feature4: 0.1499542552647165
Extracted feature5: 0.445011850513794
Extracted feature6: 0.9261430543612157
Extracted feature7: 0.7230150083759516
Extracted feature8: 0.35849041974595763
Extracted feature9: 0.26578912717333514
Extracted feature10: 0.9791491198141258
Extracted feature11: 0.5485231923896424
Extracted feature12: 0.40844712795107607
Extracted feature13: 0.3355132396361262
Extracted feature14: 0.9023689979534084
Extracted feature15: 0.8687649680728191
Extracted feature16: 0.8957148050772985
Extracted feature17: 0.677242726625785
Extracted feature18: 0.36578709216329297
Extracted feature19: 0.4119984546525355
Extracted feature20: 0.07518430004571963
Extracted feature21: 0.9962371065996352
Extracted feature22: 0.7730619646666697
Extracted feature23: 0.7521138157609484
Extracted feature24: 0.3770309449072885
Extract